In [6]:
import json

import pandas as pd
rsna_files = [
    "/Users/andrewprokhorenkov/Downloads/output_packages_open_RSNA_20220228.tsv",
    "/Users/andrewprokhorenkov/Downloads/output_packages_open_RSNA_20220308.tsv",
    "/Users/andrewprokhorenkov/Downloads/output_packages_open_RSNA_20220314.tsv",
    "/Users/andrewprokhorenkov/Downloads/output_packages_open_RSNA_20220329.tsv",
]
rsna = map(lambda f: pd.read_csv(f, sep="\t"), rsna_files)
rsna = pd.concat(rsna)
rsna_md5 = rsna["package_contents"].apply(json.loads).explode().apply(lambda x: x["hashes"]["md5sum"]).to_list()
rsna_submissions = ["RSNA_20220228", "RSNA_20220308", "RSNA_20220314", "RSNA_20220329"]

rsna_manifests_files = [
    f"/Users/andrewprokhorenkov/CTDS/projects/midrc/ssot-s3/replicated-data-rsna/{v}/imaging_data_manifest_{v}.tsv" for v in rsna_submissions
]
rsna_manifests = map(lambda f: pd.read_csv(f, sep="\t"), rsna_manifests_files)
rsna_manifests = pd.concat(rsna_manifests)
for v in rsna_submissions:
    sub_file = f"/Users/andrewprokhorenkov/CTDS/projects/midrc/ssot-s3/replicated-data-rsna/{v}/imaging_data_manifest_{v}.tsv"

    rsna_manifest = pd.read_csv(sub_file, sep="\t")

    rsna_manifest.loc[rsna_manifest["md5sum"].isin(rsna_md5), "storage_urls"].str.replace("s3://storage.ir.rsna.ai/", "", regex=False) \
        .to_csv(f"/Users/andrewprokhorenkov/Downloads/submit_{v}.lst", header=False, index=False)